In [1]:
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator

# Number of threads, nthreads = -1, means use all cores on your machine
# max_mem_size is the maximum memory (in GB) to allocate to H2O
h2o.init(nthreads = -1, max_mem_size = 8)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-macosx) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-macosx) (build 25.121-b15, mixed mode)
  Starting server from /anaconda/lib/python3.5/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/0p/99_2wyss4231_ch878lbbnqh0000gn/T/tmp04sz45yl
  JVM stdout: /var/folders/0p/99_2wyss4231_ch878lbbnqh0000gn/T/tmp04sz45yl/h2o_abbiepopa_started_from_python.out
  JVM stderr: /var/folders/0p/99_2wyss4231_ch878lbbnqh0000gn/T/tmp04sz45yl/h2o_abbiepopa_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,04 secs
H2O cluster version:,3.16.0.2
H2O cluster version age:,1 month and 8 days
H2O cluster name:,H2O_from_python_abbiepopa_djs4eh
H2O cluster total nodes:,1
H2O cluster free memory:,7.111 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [2]:
d = h2o.import_file('../data/2016_subset_all_tables.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
d.shape

(6937, 16)

In [4]:
d.head()

id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,dcoilwtico,city,state,type,cluster,transactions,holitype
7.1279e+07,2016-02-22 00:00:00,6,1.46403e+06,3,False,BEVERAGES,1148,0,31.37,Quito,Pichincha,D,13,1630,No_Holiday
8.2078e+07,2016-06-14 00:00:00,6,1.9136e+06,1,False,PERSONAL CARE,4118,0,48.49,Quito,Pichincha,D,13,1603,No_Holiday
9.28523e+07,2016-10-04 00:00:00,6,881701,2,False,GROCERY I,1042,0,48.67,Quito,Pichincha,D,13,1605,No_Holiday
7.27136e+07,2016-03-08 00:00:00,6,1.15095e+06,2,False,BEVERAGES,1124,0,36.67,Quito,Pichincha,D,13,1732,No_Holiday
7.27143e+07,2016-03-08 00:00:00,6,1.89896e+06,5,False,BEVERAGES,1126,0,36.67,Quito,Pichincha,D,13,1732,No_Holiday
8.50331e+07,2016-07-15 00:00:00,6,1.47191e+06,4,False,PET SUPPLIES,6516,0,45.93,Quito,Pichincha,D,13,1785,No_Holiday
9.65588e+07,2016-11-11 00:00:00,6,841612,9,False,CLEANING,3038,0,43.39,Quito,Pichincha,D,13,1574,No_Holiday
9.64669e+07,2016-11-10 00:00:00,6,270522,1,False,AUTOMOTIVE,6806,0,44.62,Quito,Pichincha,D,13,1497,No_Holiday
7.21284e+07,2016-03-02 00:00:00,6,2.0104e+06,1,False,GROCERY I,1072,0,34.57,Quito,Pichincha,D,13,1817,No_Holiday
7.41329e+07,2016-03-23 00:00:00,6,1.95466e+06,16.636,True,PRODUCE,2008,1,38.28,Quito,Pichincha,D,13,1850,No_Holiday


In [5]:
#to start, lets go with on_promotion, transactions, and family since those were best last time
x = ['onpromotion', 'transactions', 'family']
y = 'unit_sales'

rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', seed=1)
rf_fit1.train(x=x, y=y, training_frame = d)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [6]:
rf_fit1.model_performance()


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 323.13404382020303
RMSE: 17.975929567624675
MAE: 6.446356345737289
RMSLE: 0.8938706956256336
Mean Residual Deviance: 323.13404382020303


In [7]:
#granted, it's not really taking advantage of the random forest to only feed it three features
#but this is kinda a proof of concept!

In [8]:
test = h2o.import_file('../data/test_concatted.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
pred = rf_fit1.predict(test)

drf prediction progress: |████████████████████████████████████████████████| 100%


/anaconda/lib/python3.5/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'family' has levels not trained on: [HOME APPLIANCES]
  warnings.warn(w)


In [10]:
out = test[['id']].cbind(pred)

In [11]:
out.columns = ['id', 'unit_sales']

In [12]:
out_df = out.as_data_frame()

In [13]:
out_df.to_csv('../data/3f_rf_pred.csv', index = False)

In [14]:
test.head()

C1,id,store_nbr,onpromotion,family,transactions
0,1.25497e+08,1,False,GROCERY I,1523.84
1,1.27395e+08,1,False,DAIRY,1523.84
2,1.25708e+08,1,False,GROCERY I,1523.84
3,1.27605e+08,1,False,DAIRY,1523.84
4,1.25918e+08,1,False,GROCERY I,1523.84
5,1.27816e+08,1,False,DAIRY,1523.84
6,1.26129e+08,1,False,GROCERY I,1523.84
7,1.28027e+08,1,False,DAIRY,1523.84
8,1.2634e+08,1,False,GROCERY I,1523.84
9,1.28237e+08,1,False,DAIRY,1523.84
